Mount Google Drive with Colab

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/My Drive/RetinaNet/')
!git clone https://github.com/fizyr/keras-retinanet.git

In [ ]:
!nvidia-smi

In [ ]:
import os
! git clone https://github.com/NVIDIA/apex
os.chdir('apex')
! pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./
os.chdir('../')

In [ ]:
%cd '/content/drive/My Drive/RetinaNet/keras-retinanet/'

!pip install .

/content/drive/My Drive/RetinaNet/keras-retinanet
Processing /content/drive/My Drive/RetinaNet/keras-retinanet
  Created wheel for keras-retinanet: filename=keras_retinanet-1.0.0-cp36-cp36m-linux_x86_64.whl size=168074 sha256=6e883b671eba475c89c69799a92ed51144693300119eb621d41bda268d6a1b3a
  Stored in directory: /root/.cache/pip/wheels/3e/e0/41/6e7d027332a7e36343aa0047e7425e376aa855619dd7afad9a
  Created wheel for keras-resnet: filename=keras_resnet-0.2.0-py2.py3-none-any.whl size=20487 sha256=3cd55b098449f9f60877af307f229f9d4872d1ea5d34bb3a9c9bae9c287afb5f
  Stored in directory: /root/.cache/pip/wheels/5f/09/a5/497a30fd9ad9964e98a1254d1e164bcd1b8a5eda36197ecb3c
Successfully built keras-retinanet keras-resnet


In [ ]:
!python setup.py build_ext --inplace

running build_ext
skipping 'keras_retinanet/utils/compute_overlap.c' Cython extension (up-to-date)
copying build/lib.linux-x86_64-3.6/keras_retinanet/utils/compute_overlap.cpython-36m-x86_64-linux-gnu.so -> keras_retinanet/utils


In [ ]:
import os
import shutil
import zipfile
import urllib
import xml.etree.ElementTree as ET
import numpy as np
import csv
import pandas
from google.colab import drive
from google.colab import files

# Training Model

Download pretrained model and run training.

In the next cell choose one option:

1.   download Fizyr Resnet50 pretrained model
2.   download your custom pretrained model, to continue previous training epochs

In the last cell optionally export trained model to Google Drive.


In [ ]:
import urllib.request

In [ ]:
PRETRAINED_MODEL = './snapshots/_pretrained_model.h5'

#### OPTION 1: DOWNLOAD INITIAL PRETRAINED MODEL FROM FIZYR ####
URL_MODEL = 'https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5'
urllib.request.urlretrieve(URL_MODEL, PRETRAINED_MODEL)

#### OPTION 2: DOWNLOAD CUSTOM PRETRAINED MODEL FROM GOOGLE DRIVE. CHANGE DRIVE_MODEL VALUE. USE THIS TO CONTINUE PREVIOUS TRAINING EPOCHS ####
# drive.mount('/content/gdrive')
# DRIVE_MODEL = '/content/drive/My Drive/RetinaNet/keras-retinanet/snapshots/resnet50_csv_01.h5'
# shutil.copy(DRIVE_MODEL, './snapshots/resnet50_csv_01.h5')


print('Downloaded pretrained model to ' + PRETRAINED_MODEL)

Downloaded pretrained model to ./snapshots/_pretrained_model.h5


In [ ]:
%cd '/content/drive/My Drive/RetinaNet/keras-retinanet/'

/content/drive/My Drive/RetinaNet/keras-retinanet


In [ ]:
%cd keras_retinanet/bin/
!chmod +x train.py

/content/drive/My Drive/RetinaNet/keras-retinanet/keras_retinanet/bin


In [ ]:
!sudo chmod -R 755 /usr/bin/env

In [ ]:
%cd '/content/drive/My Drive/RetinaNet/keras-retinanet/'

/content/drive/My Drive/RetinaNet/keras-retinanet


In [ ]:
!keras_retinanet/bin/train.py --freeze-backbone --random-transform --weights snapshots/resnet50_csv_09.h5 --batch-size 8 --steps 338 --epochs 50 csv aug_train.csv class.csv

2020-12-01 20:20:15.222430: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Creating model, this may take a second...
2020-12-01 20:20:32.033695: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-12-01 20:20:32.103257: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-12-01 20:20:32.103876: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.73GiB deviceMemoryBandwidth: 298.08GiB/s
2020-12-01 20:20:32.103921: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-12-01 20:20:32.327994: I t

In [ ]:
%cd keras_retinanet/bin/
!chmod +x convert_model.py

/content/drive/My Drive/RetinaNet/keras-retinanet/keras_retinanet/bin


In [ ]:
!keras_retinanet/bin/convert_model.py snapshots/resnet50_csv_23.h5 inference/inference.h5

In [ ]:
%cd keras_retinanet/bin/
!chmod +x evaluate.py

/content/drive/My Drive/RetinaNet/keras-retinanet/keras_retinanet/bin


In [ ]:
!keras_retinanet/bin/evaluate.py --iou-threshold 0.5 --score-threshold 0.1 --save-path results/ csv valid.csv class.csv inference/inference.h5

# Inference
Run inference with uploaded image on trained model.

In [ ]:
THRES_SCORE = 0.2

In [ ]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%reload_ext autoreload
%autoreload 2

# import keras
import keras
# import keras.backend
# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
from keras import backend

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time

# set tf backend to allow memory to grow, instead of claiming everything
import tensorflow as tf
# from keras.backend.tensorflow_backend import set_session
# import tensorflow_backend

def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

# use this environment flag to change which GPU to use
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# set the modified tf session as backend in keras
# keras.backend.tensorflow_backend.set_session(get_session())

In [ ]:
model_path = os.path.join('snapshots', 'resnet50_csv_23.h5')
print(model_path)

# load retinanet model
model = models.load_model(model_path, backbone_name='resnet50')
model = models.convert_model(model)

# load label to names mapping for visualization purposes
labels_to_names = pandas.read_csv('class.csv',header=None).T.loc[0].to_dict()

snapshots/resnet50_csv_23.h5


In [ ]:
import pandas as pd
df = pd.DataFrame()

In [ ]:
def img_inference(img_path,df):
  image = read_image_bgr(img_infer)
  print(img_infer)
  # copy to draw on
  draw = image.copy()
  draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

  # preprocess image for network
  image = preprocess_image(image)
  image, scale = resize_image(image)

  # process image
  start = time.time()
  boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
  print("processing time: ", time.time() - start)

  # correct for image scale
  boxes /= scale

  # visualize detections
  for box, score, label in zip(boxes[0], scores[0], labels[0]):
      # scores are sorted so we can break
      if score < THRES_SCORE:
          break

      color = label_color(label)

      b = box.astype(int)
      row = pd.Series([img_infer,score,b[0],b[1],b[2],b[3],labels_to_names[label]])
      row_df = pd.DataFrame([row])
      df = pd.concat([df,row_df], ignore_index=True)
      draw_box(draw, b, color=color)

      caption = "{} {:.3f}".format(labels_to_names[label], score)
      print(caption)
      draw_caption(draw, b, caption)

  plt.figure(figsize=(10, 10))
  plt.axis('off')
  plt.imshow(draw)
  plt.savefig('/content/drive/My Drive/RetinaNet/keras-retinanet/output image/'+ img_infer)
  plt.show()
  return df

In [ ]:
from google.colab import files
uploaded = files.upload()
for i in range(500):
  img_infer = list(uploaded)[i]

  print('Running inference on: ' + img_infer)
  df = img_inference(img_infer,df)

In [ ]:
print(df)

In [ ]:
df.to_csv("retinanet.csv",index=False)